<a href="https://colab.research.google.com/github/Giraud-Pierre/PINN_for_SEDMES/blob/adsorption_exercise/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import scipy.io

In [ ]:
np.random.seed(0)
tf.random.set_seed(1234)

In [ ]:
class PhysicsInformedNN:
  '''PINN model tailored to answer the adsorption exercise'''
  def __init__(self,L,ug,epsb,c0,kg,Ke, layers):
    '''object constructor (initialize object at creation). Takes the folowing parameters:
    L the length of the bed (m)
    ug the linear gas velocity (m/s)
    epsb the bed porosity (dimensionless)
    c0 the concentration of pollution of the gas (kmol/m3)
    kg mass transfer coefficient of the gas to the particles( m/s)
    Ke the equilibirum constant (dimentionless)
    layers an array containing the number of hidden layers and neurons per layer'''

    #initialize constants
    self.L = L
    self.ug = ug
    self.epsb = epsb
    self.c0 = c0
    self.kg = kg
    self. Ke = Ke

    #initialize NN
    self.weights, self.biases = self.initialize_NN(layers)

  def initialize_NN(layers):
    '''return initial weights and biases for a feed forward neural network 
    with a given number of layers and neurons per layer'''
    weights = []
    biases = []
    num_layers = len(layers)
    for i in range(num_layers -1):
      #create a set of defaults weights and biases between each layer
      in_dim = layers[i]
      out_dim = layers[i+1]
      xavier_stddev = np.sqrt(2/(in_dim + out_dim))
            #initialize the weights using Xavier initialization to avoid problems such as vanishing or exploding gradients
      W = tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev = xavier_stddev), dtype = tf.float32) 
            #initialize biases at 0
      b = tf.Variable(tf.zeros([1,layers[i+1]], dtype = tf.float32), dtype = tf.float32)
      weights.append(W)
      biases.append(b)
    return weights, biases

